<a href="https://colab.research.google.com/github/44REAM/radianet/blob/master/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:

!pip install optuna
!pip install pytorch_lightning
!pip install efficientnet_pytorch

%cd '/content/drive/My Drive/'
!git clone https://github.com/44REAM/radianet.git
%cd radianet


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=ba576dd850342a1ec7bd1532a9ca9484893aaadff7f840d5c0da8aee54cddf04
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch
/content/drive/My Drive
fatal: destination path 'radianet' already exists and is not an empty directory.
/content/drive/My Drive/radianet


In [4]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
import sys
import os
sys.path.append('..')

import torch
import optuna
import pytorch_lightning as pl

from radianet import get_dataloader
from radianet.datasets import SampleDataset2D, LIDCDataset, Transforms
from radianet.models import Simple3DCNN, MyEfficientNet
from radianet.callbacks import MetricsCallback
from radianet.lightningnet import LightningNet


In [0]:
class Config():
    BATCHSIZE = 16
    N_TRIALS = 1
    EPOCHS = 100
    N_CLASSES = 1
    BORDER_MODE = 0
    NAME = 'lidc'
    IMAGE_SIZE = 240
    LIDC_PATH = '/content/drive/My Drive/dataset/radiology/TCIA_LIDC-IDRI/preprocess/'
    EFFICIENTNET_B0_LAYER = 15

    MODEL_DIR = './checkpoints/' + NAME
    DB_NAME = 'sqlite:///' + NAME + '.db'


In [18]:

def objective(trial):
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(Config.MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), 
        monitor="val_loss", save_top_k = -1
    )
    
    metrics_callback = MetricsCallback()

    trainer = pl.Trainer(
        logger=False,
        max_epochs=Config.EPOCHS,
        checkpoint_callback=checkpoint_callback,
        callbacks=[metrics_callback],
        gpus=0 if torch.cuda.is_available() else None
    )
    
    transforms = Transforms()
    datasets = LIDCDataset(transforms, Config.LIDC_PATH)
    #datasets = SampleDataset2D(transforms, n_sample = 10)
    dataloader = get_dataloader(datasets, Config.BATCHSIZE)

    lightning_model = LightningNet(trial, Config, MyEfficientNet, dataloader)
    print("before git")
    trainer.fit(lightning_model)
    
    print("here")
    return metrics_callback.metrics[-1]["val_loss"]

study = optuna.create_study(study_name = Config.NAME, storage = Config.DB_NAME, load_if_exists = True)
study.optimize(objective, n_trials= Config.N_TRIALS)


[I 2020-05-12 06:32:48,556] Using an existing study with name 'test_chechpoint' instead of creating a new one.
INFO:lightning:GPU available: False, used: False
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0
before git


INFO:lightning:
    | Name                                                 | Type                    | Params
---------------------------------------------------------------------------------------------
0   | model                                                | MyEfficientNet          | 5 M   
1   | model.head                                           | Sequential              | 928   
2   | model.head.0                                         | Conv2dStaticSamePadding | 864   
3   | model.head.0.static_padding                          | ZeroPad2d               | 0     
4   | model.head.1                                         | BatchNorm2d             | 64    
5   | model.body_no_grad                                   | Sequential              | 2 M   
6   | model.body_no_grad.0                                 | MBConvBlock             | 1 K   
7   | model.body_no_grad.0._depthwise_conv                 | Conv2dStaticSamePadding | 288   
8   | model.body_no_grad.0._depthwise_conv.s

linear input not have been calculate


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning:

The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.



...

/content/drive/My Drive/radianet/radianet/lightningnet.py:49: UserWarning:

Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.



torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])
torch.Size([2, 1])


INFO:lightning:Detected KeyboardInterrupt, attempting graceful shutdown...
[W 2020-05-12 06:33:12,913] Setting status of trial#2 as TrialState.FAIL because of the following error: IndexError('list index out of range',)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 699, in _run_trial
    result = func(trial)
  File "<ipython-input-18-e7d03db2dcc5>", line 30, in objective
    return metrics_callback.metrics[-1]["val_loss"]
IndexError: list index out of range



here


IndexError: ignored

In [0]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))